# TP 7: Classification multi-classe

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import *

## Importer le jeu de données `digits`

In [5]:
from sklearn import datasets

Charger le jeu de données `digits` via la fonction `datasets.load_digits` dans un nom `digits`. C'est un objet `Bunch` qui ressemble à un dictionnaire. Vous pouvez afficher les clefs d'indice avec `digits.keys()` et regardez la description de ce jeu de données en affichant par `print` le contenu associé à la clef `DESCR` (grâce à `digits['DESCR']` ou de façon équivalente via `digits.DESCR`).

In [6]:
digits = datasets.load_digits()
print(digits.DESCR)

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

**Question** Nommez `X_digits` et `y_digits` les valeurs associées aux clefs d'indice `data` et `target` qui correspondent aux données et aux valeur associées (les classes ou cibles).

In [7]:
X_digits = digits.data
y_digits = digits.target
y_digits, X_digits.shape

(array([0, 1, 2, ..., 8, 9, 8]), (1797, 64))

La clef "images" est une liste d'images que l'on peut afficher. Voici les 50 premières :

In [8]:
nrows, ncols = 5, 10
fig, ax = plt.subplots(nrows, ncols, figsize=(10, 5))
for i in range(nrows*ncols):
    ax[i//ncols,i%ncols].imshow(digits.images[i], cmap=plt.cm.gray_r)

**Question**: Découpez l'échantillon en `X_train`, `y_train`, `X_test`, `y_test` en gardant les 100 derniers exemples pour le test.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, test_size=100, random_state=1)




Vérifiez que vous avez bien les dimensions suivantes pour ces objets : ((1697, 64), (100, 64), (1697,), (100,))

**Question** affichez les dimensions des matrices correspondant aux 4 échantillons.

In [10]:
((len(X_train),len(X_train[0])),(len(X_test),len(X_test[0])),(len(y_train),),(len(y_test),))

((1697, 64), (100, 64), (1697,), (100,))

## Perceptron multiclasse "Un-contre-Tous"

**Question**: Observez le nombre de classes et etudier leur distribution dans `train` et dans le `test`. 

In [11]:
classes_list = np.unique(y_digits).astype(int)
print("Nombre de classe:",len(classes_list),"\n")
#nombre de classe est 10
Xi_mean_train = [np.mean(X_train[y_train == cls]) for cls in classes_list]
Xi_var_train = [np.var(X_train[y_train == cls]) for cls in classes_list]
print("#### Train ####\n")
print("Moyenne des x pour chaque classe (par ordre) :",Xi_mean_train,"\n")
print("Variance des x pour chaque classe (par ordre) :",Xi_mean_train)
Xi_mean_test = [np.mean(X_train[y_train == cls], axis=0) for cls in classes_list]
Xi_var_test = [np.var(X_train[y_train == cls], axis=0) for cls in classes_list]
print("\n#### Test ####\n")
print("Moyenne des x pour chaque classe (par ordre) :",Xi_mean_train,"\n")
print("Variance des x pour chaque classe (par ordre) :",Xi_mean_train)

Nombre de classe: 10 

#### Train ####

Moyenne des x pour chaque classe (par ordre) : [4.950785928143713, 4.872441520467836, 4.899242424242424, 4.791061046511628, 4.865477071005917, 4.807839912280702, 4.869703389830509, 4.75217803030303, 5.145425451807229, 4.894306752873563] 

Variance des x pour chaque classe (par ordre) : [4.950785928143713, 4.872441520467836, 4.899242424242424, 4.791061046511628, 4.865477071005917, 4.807839912280702, 4.869703389830509, 4.75217803030303, 5.145425451807229, 4.894306752873563]

#### Test ####

Moyenne des x pour chaque classe (par ordre) : [4.950785928143713, 4.872441520467836, 4.899242424242424, 4.791061046511628, 4.865477071005917, 4.807839912280702, 4.869703389830509, 4.75217803030303, 5.145425451807229, 4.894306752873563] 

Variance des x pour chaque classe (par ordre) : [4.950785928143713, 4.872441520467836, 4.899242424242424, 4.791061046511628, 4.865477071005917, 4.807839912280702, 4.869703389830509, 4.75217803030303, 5.145425451807229, 4.894306

Nous sommes donc face à un problème **multiclasse**, ou le nombre de classes est *k=10*. Une technique pour traiter ce type de problèmes consiste à le reduire en *k* problemes binaires. Specifiquement, on construit un classifieur binaire pour chaque classe: tous les exemples $x_i$ associes a cette classe recoivent une etiquette $+1$, alors que les exemples des autres classes sont etiquettes $-1$. Ceci correspond a un codage binaire de chaque classe: elle est associee a un vecteur $z$ de taille $k$, tel que $z_i = +1$ if $y_i = k$ et $z_i = -1$ autrement. 

Cette technique s'appelle le **one-versus-all**. 



### Codage binaire des labels

**Question**:  Concretement, on cree donc un vecteur binaire "cible" $y\in\{−1,+1\}^N$ (ou $N$ est le nombre d'exemples) que l'on associe a chaque classe (via un dictionnaire) pour chaque jeu de donnees `train` et `test`. 

In [12]:
donnees=[y_train,y_test]
assoc=["train","test"]
dic={}
res={}

for d in donnees:#pour chaque vecteur y (y_train,y_test)
    dic={}
    for k in range (len(classes_list)):#pour chaque classe
        z=[]
        for y in range(len(d)):#pour chaque yi de (y_train, y_test)
            if k==d[y]:# tester si la valeur de yi est egale a k 
                z=z+[1]
            else:
                z=z+[-1]
    
        dic[k]=z #Ajouter le vecteur z a la table de hashage  (classe:z)
    res[assoc[donnees.index(d)]]=dic # Hashé le nom de jeu de donnees avec le dictionnaire {classe:vecteur}

    

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


**Question**: On entraine ensuite $10$ classifieurs binaires, dans ce cas-ci un perceptron, que l'on associe chacune des classes. Utilisez l'implementation du perceptron fournie dans `sklearn`. Fixez le nombre d'iterations maximum a $1$, $10$, puis $100$).  

In [13]:
classePerceprton={}
for c in range(len(classes_list)):
    perc=Perceptron(max_iter=100)
    perc.fit(X_train,res["train"][c])
    classePerceprton[c]=perc

### Prediction multiclasse

**Question**: On cherche a present a predire une des $k$ classes, en utilisant la regle de decision suivante, ou $\bf{w}_y$ est le vecteur de parametres appris avec le perceptron binaire pour la classe $y$:

$$
\hat{y} = \text{argmax}_{y\in\{1,\ldots,k\}} \bf{w}_y^{\top} \bf{x}
$$
Predisez les etiquettes multiclasses successivement sur les donnees `train` et `test`. 

In [14]:
#trouver le y qui donne la valeur maximale pour W𝐲 ⊤ 𝐱 (pour chaque x)
def predict_labels(X):
    y_hat=[]
    for i in range (len(X)):
        max=sum(X[i]*classePerceprton[0].coef_[0])
        y=0
        for j in range(1,len(classes_list)):
            if sum(X[i]*classePerceprton[j].coef_[0])>max:
                max=sum(X[i]*classePerceprton[j].coef_[0])
                y=j
        y_hat.append(y)
    return y_hat

y_hat_train=predict_labels(X_train)
y_hat_test=predict_labels(X_test)

### Evaluation 

**Question:** Calculer le taux d'erreur (cad, nombre de prediction correctes/nombre d'exemples) sur `train` et `test`. 

In [15]:
error=list((y_hat_train==y_train)).count(True)/len(X_train)
print(" Taux d'erreur pour train :",error)

error=list((y_hat_test==y_test)).count(True)/len(X_test)
print(" Taux d'erreur pour test :",error)


 Taux d'erreur pour train : 0.9840895698291102
 Taux d'erreur pour test : 0.97


**Question**: Evaluer egalement votre modele de classification multi-classe en termes de **precision (P)**, **rappel (R)** et **F1-score (F1)**:

$$
P = \frac{TP}{TP+FP} \\
R = \frac{TP}{TP+FN}\\
F1 = 2 \frac{P\times R}{P+R}
$$
ou TP designent les vrais positifs, FP les faux positifs, et FN les faux negatifs. Ces mesures sont fournies dans `sklearn.metrics`. Utiliser l'option `average='macro'`.



In [16]:
print("precision (P):",precision_score(y_test, y_hat_test, average='macro'))
print("rappel (R):",recall_score(y_test, y_hat_test, average='macro') )
print("F1-score (F1):",f1_score(y_test, y_hat_test, average='macro'))

precision (P): 0.9765501165501165
rappel (R): 0.9651515151515152
F1-score (F1): 0.9696080011942081


**Question:** En vue de visualiser les erreurs faites pour le modele, calculer la **matrice de confusion**, egalement disponible dans `sklearn.matrix`. Analyser cette matrice. Quelles sont les erreurs les plus frequentes? Tentez d'expliquer. 

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_hat_train,labels=[0, 1, 2,3,4,5,6,7,8,9])

#Les erreurs les plus frequantes sont souvent sur les classes 8 et 9


array([[167,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 167,   0,   2,   0,   0,   2,   0,   0,   0],
       [  0,   0, 165,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 170,   0,   1,   0,   1,   0,   0],
       [  0,   1,   0,   0, 167,   0,   0,   0,   1,   0],
       [  0,   0,   0,   0,   0, 171,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 177,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 165,   0,   0],
       [  0,   5,   2,   1,   1,   1,   1,   0, 155,   0],
       [  0,   0,   0,   0,   1,   0,   0,   2,   5, 166]])